## Read in word list

In [1]:
import src.utils.git as gitutil
bpath = gitutil.get_root()

In [2]:
word_list_name = 'twl06.txt'

word_list_file = bpath + '/data/processed/' + word_list_name

with open(word_list_file, 'r') as f:
    word_list = f.read().splitlines()

## Initialise

In [3]:
possible_words = word_list

## Read in pre-calculated full cross-check

In [4]:
import numpy as np

In [5]:
full_cross_check = np.load(bpath + '/data/processed/full_cross_check.npy')

### Retrieve diagonals to get correct positions

In [6]:
diags = np.diagonal(full_cross_check, axis1=2, axis2=3)

### Determine remaining letters in incorrect positions

Take row maxes excluding diagonals

In [7]:
rowmaxes = np.logical_and((np.tril(full_cross_check, -1) + np.triu(full_cross_check, 1)).max(axis=3), ~diags)

In [8]:
possible_words[3801]

'irate'

In [9]:
np.unique(rowmaxes[3801], axis=0, return_counts=True)

(array([[False, False, False, False, False],
        [False, False, False, False,  True],
        [False, False, False,  True, False],
        [False, False, False,  True,  True],
        [False, False,  True, False, False],
        [False, False,  True, False,  True],
        [False, False,  True,  True, False],
        [False, False,  True,  True,  True],
        [False,  True, False, False, False],
        [False,  True, False, False,  True],
        [False,  True, False,  True, False],
        [False,  True, False,  True,  True],
        [False,  True,  True, False, False],
        [False,  True,  True, False,  True],
        [False,  True,  True,  True, False],
        [False,  True,  True,  True,  True],
        [ True, False, False, False, False],
        [ True, False, False, False,  True],
        [ True, False, False,  True, False],
        [ True, False, False,  True,  True],
        [ True, False,  True, False, False],
        [ True, False,  True, False,  True],
        [ 

In [10]:
np.unique(diags[3801], axis=0, return_counts=True)

(array([[False, False, False, False, False],
        [False, False, False, False,  True],
        [False, False, False,  True, False],
        [False, False, False,  True,  True],
        [False, False,  True, False, False],
        [False, False,  True, False,  True],
        [False, False,  True,  True, False],
        [False, False,  True,  True,  True],
        [False,  True, False, False, False],
        [False,  True, False, False,  True],
        [False,  True, False,  True, False],
        [False,  True, False,  True,  True],
        [False,  True,  True, False, False],
        [False,  True,  True, False,  True],
        [False,  True,  True,  True, False],
        [False,  True,  True,  True,  True],
        [ True, False, False, False, False],
        [ True, False, False, False,  True],
        [ True, False,  True, False, False],
        [ True, False,  True, False,  True],
        [ True, False,  True,  True, False],
        [ True,  True, False, False, False],
        [ 